In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import warnings

from Model_functions import *

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.metrics import r2_score
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pandas.core.common import SettingWithCopyWarning

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

warnings.simplefilter(action = 'ignore', category=SettingWithCopyWarning)
pd.set_option('display.max_columns', None, 'display.max.rows', None)


2022-09-12 15:50:19.481118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
per_game_data = pd.read_csv("data/MVP_per_game.csv")
#per_possession_data = pd.read_csv("data/MVP_per_possession.csv")

In [3]:
test_year = 2022
years = [year for year in range(1980, 2022)]

historical_data = per_game_data[per_game_data["Year"] != test_year]
final_year_data = per_game_data[per_game_data["Year"] == test_year]

In [4]:
#List of important features extracted in MVP_predictions.ipynb

feats = ['PER', 'WS/48', 'VORP', 'BPM', 'OBPM', 'OWS', 'PTS', 'USG%',
       'W/L%', 'TS%', 'TOV', 'DWS', 'FTr', 'DRB%']

id_cols = ['Rank','Player', 'Age', 'Tm', 'Year', 'Share']

historical_data_reduced = historical_data[id_cols + feats]
final_year_data_reduced = final_year_data[feats]

id_train = historical_data[['Rank','Player', 'Age', 'Tm', 'Year']]
id_test = final_year_data[['Rank','Player', 'Age', 'Tm', 'Year']]

y_train = historical_data['Share']
y_test = final_year_data['Share']

x_train = historical_data[feats]
x_test= final_year_data[feats]

#to_drop_train = ['Rank','Player', 'Age', 'Tm', 'Year']

## Hyperparameter Tuning for Logistic Regression

## Hyperparameter Tuning for the Neural Network

In [6]:
model_NN = KerasRegressor(build_fn=keras_model, verbose=0)

estimator = Pipeline([('scaler', StandardScaler()), 
                      ("model", model_NN)])

param_grid = {'model__epochs': [50, 80],
              'model__n1': [40, 70],
              #'model__n2': [20,30],
              'model__batch_size':[50, 70],
              'model__dropout': [0.1, 0.2, 0.3],
              'model__optimizer': [keras.optimizers.SGD, keras.optimizers.Adam] ,
              'model__learn_rate': [0.15, 0.2]#,
              #'model__momentum': [0.0, 0.3, 0.7]
             }

grid_NN = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1, cv=5, verbose=1)

/var/folders/_d/d5z6h7kx6ws38vzmkgr9pss40000gp/T/ipykernel_8768/3085022073.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_NN = KerasRegressor(build_fn=keras_model, verbose=0)


In [7]:
grid_NN.fit(x_train,y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


2022-09-12 12:14:26.614177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-12 12:14:26.614177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-12 12:14:26.618438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        <keras.wrappers.scikit_learn.KerasRegressor object at 0x12f765c70>)]),
             n_jobs=-1,
             param_grid={'model__batch_size': [50, 70],
                         'model__dropout': [0.1, 0.2, 0.3],
                         'model__epochs': [50, 80],
                         'model__learn_rate': [0.15, 0.2],
                         'model__n1': [40, 70],
                         'model__optimizer': [<class 'keras.optimizers.optimizer_v2.gradient_descent.SGD'>,
                                              <class 'keras.optimizers.optimizer_v2.adam.Adam'>]},
             verbose=1)

In [ ]:
grid_NN.best_params_

{'model__batch_size': 50,
 'model__dropout': 0.1,
 'model__epochs': 50,
 'model__learn_rate': 0.2,
 'model__n1': 40,
 'model__optimizer': keras.optimizers.optimizer_v2.gradient_descent.SGD}

In [9]:
# param_grid2 = {'model__epochs': [60, 70, 80],
#               'model__n1': [30,40],
#               'model__n2': [30,40],
#               'model__batch_size':[40, 50, 60],
#               'model__dropout': [0.15, 0.2, 0.25],
#               'model__optimizer': [keras.optimizers.SGD],#['RMSprop', 'Adam', 'sgd'],
#               'model__learn_rate': [0.12, 0.15, 0.18]
#              }

param_grid2 = {'model__epochs': [50, 60],
              'model__n1': [40, 50],
              #'model__n2': [30,40],
              'model__batch_size':[50, 60],
              'model__dropout': [0.1, 0.12, 0.15],
              'model__optimizer': [keras.optimizers.SGD],#['RMSprop', 'Adam', 'sgd'],
              'model__learn_rate': [0.18, 0.2, 0.22]
             }

grid_NN2 = GridSearchCV(estimator=estimator, param_grid=param_grid2, n_jobs=-1, cv=5, verbose=1)

In [10]:
grid_NN2.fit(x_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


2022-09-12 12:28:33.115962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-12 12:28:33.115964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-12 12:28:33.118751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        <keras.wrappers.scikit_learn.KerasRegressor object at 0x12f765c70>)]),
             n_jobs=-1,
             param_grid={'model__batch_size': [50, 60],
                         'model__dropout': [0.1, 0.12, 0.15],
                         'model__epochs': [50, 60],
                         'model__learn_rate': [0.18, 0.2, 0.22],
                         'model__n1': [40, 50],
                         'model__optimizer': [<class 'keras.optimizers.optimizer_v2.gradient_descent.SGD'>]},
             verbose=1)

In [11]:
grid_NN2.best_params_

{'model__batch_size': 60,
 'model__dropout': 0.1,
 'model__epochs': 60,
 'model__learn_rate': 0.22,
 'model__n1': 50,
 'model__optimizer': keras.optimizers.optimizer_v2.gradient_descent.SGD}

In [12]:
# param_grid3 = {'model__epochs': [70, 100],
#               'model__n1': [30,40],
#               'model__n2': [40,50],
#               'model__batch_size':[40, 50],
#               'model__dropout': [0.15, 0.18],
#               'model__optimizer': [keras.optimizers.SGD],#['RMSprop', 'Adam', 'sgd'],
#               'model__learn_rate': [0.15]
#              }

param_grid3 = {'model__epochs': [60, 70],
              'model__n1': [50, 60],
              #'model__n2': [40,50],
              'model__batch_size':[60],
              'model__dropout': [0.1, 0.13],
              'model__optimizer': [keras.optimizers.SGD],#['RMSprop', 'Adam', 'sgd'],
              'model__learn_rate': [0.2, 0.22, 0.25]
             }

grid_NN3 = GridSearchCV(estimator=estimator, param_grid=param_grid3, n_jobs=-1, cv=5, verbose=1)

In [13]:
grid_NN3.fit(x_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        <keras.wrappers.scikit_learn.KerasRegressor object at 0x12f765c70>)]),
             n_jobs=-1,
             param_grid={'model__batch_size': [60],
                         'model__dropout': [0.1, 0.13],
                         'model__epochs': [60, 70],
                         'model__learn_rate': [0.2, 0.22, 0.25],
                         'model__n1': [50, 60],
                         'model__optimizer': [<class 'keras.optimizers.optimizer_v2.gradient_descent.SGD'>]},
             verbose=1)

In [14]:
grid_NN3.best_params_

{'model__batch_size': 60,
 'model__dropout': 0.13,
 'model__epochs': 60,
 'model__learn_rate': 0.25,
 'model__n1': 50,
 'model__optimizer': keras.optimizers.optimizer_v2.gradient_descent.SGD}

In [17]:
# param_grid4 = {'model__epochs': [80, 100],
#               'model__n1': [40,50],
#               'model__n2': [50,60],
#               'model__batch_size':[40],
#               'model__dropout': [0.15, 0.18],
#               'model__optimizer': [keras.optimizers.SGD],#['RMSprop', 'Adam', 'sgd'],
#               'model__learn_rate': [0.15]
#              }

# grid_NN4 = GridSearchCV(estimator=estimator, param_grid=param_grid4, n_jobs=-1, cv=5, verbose=1)

In [19]:
grid_NN4.fit(x_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        <keras.wrappers.scikit_learn.KerasRegressor object at 0x13421ba00>)]),
             n_jobs=-1,
             param_grid={'model__batch_size': [40],
                         'model__dropout': [0.15, 0.18],
                         'model__epochs': [80, 100],
                         'model__learn_rate': [0.15], 'model__n1': [40, 50],
                         'model__n2': [50, 60],
                         'model__optimizer': [<class 'keras.optimizers.optimizer_v2.gradient_descent.SGD'>]},
             verbose=1)

In [20]:
grid_NN4.best_params_

{'model__batch_size': 40,
 'model__dropout': 0.15,
 'model__epochs': 80,
 'model__learn_rate': 0.15,
 'model__n1': 40,
 'model__n2': 60,
 'model__optimizer': keras.optimizers.optimizer_v2.gradient_descent.SGD}

## Hyperparameter Tuning for the Random Forest Regressor

In [5]:
#Hyperparameter tuning for the Random Forest model

forest = RandomForestRegressor(random_state=42)

#number of trees
n_estim = [int(x) for x in np.linspace(10, 50, num = 9)]
# maximum number of levels allowed in each decision tree
max_depth = [int(x) for x in np.linspace(10, 30, num = 5)]
# minimum sample number to split a node
min_samples_split = [2, 6, 8]
# minimum sample number that can be stored in a leaf node
min_samples_leaf = [1, 3, 4, 7]
# method used to sample data points. All initial random searches select True
#bootstrap = [True]


random_grid = {"n_estimators": n_estim, 
               "max_depth": max_depth,
               "min_samples_split": min_samples_split,
               "min_samples_leaf": min_samples_leaf#,
               #"bootstrap": bootstrap
              }

rf_random = RandomizedSearchCV(estimator = forest,
                               param_distributions = random_grid,
                               n_iter = 100,
                               cv = 5, 
                               verbose = 1, 
                               n_jobs = -1,
                               random_state = 42
                              )

rf_random.fit(x_train, y_train)
print(rf_random.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'n_estimators': 45, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 20}


In [6]:
#Using the best model from the random search, we look into closer
#ranges of hyperparameters using a grid search


random_grid = {"n_estimators": [int(x) for x in np.linspace(30, 50, num = 5)], 
               "max_depth": [15, 18, 20, 23, 25],
               "min_samples_split": [2,3,4],
               "min_samples_leaf": [2, 3, 4]
              }

rf_random = GridSearchCV(estimator = forest,
                               param_grid = random_grid,
                               cv = 5, 
                               verbose = 1, 
                               n_jobs = -1
                              )

rf_random.fit(x_train, y_train)
print(rf_random.best_params_)


Fitting 5 folds for each of 225 candidates, totalling 1125 fits
{'max_depth': 18, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 45}


In [8]:
#Using the best model from the random search, we look into closer
#ranges of hyperparameters using a grid search


random_grid = {"n_estimators": [42,43,44,45,46,47,47], 
               "max_depth": [16,17,18,19],
               "min_samples_split": [2,3],
               "min_samples_leaf": [2,3,4]
              }

rf_random = GridSearchCV(estimator = forest,
                               param_grid = random_grid,
                               cv = 5, 
                               verbose = 1, 
                               n_jobs = -1
                              )

rf_random.fit(x_train, y_train)
print(rf_random.best_params_)


Fitting 5 folds for each of 168 candidates, totalling 840 fits
{'max_depth': 17, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 46}


## Hyperparameter tuning for XGB Regressor

In [14]:
 #number of trees used in sequence
trees_grid = [20,23,27,32,36,40,45,50]
# value of learning rate used by gradient descent
alpha_grid = [0.16,0.18,0.21,0.25]
#maximum depth allowed for each tree
max_depth_grid = [2,3,4]


xgb_grid = {'n_estimators': trees_grid,
            'learning_rate': alpha_grid,
            'max_depth': max_depth_grid
            }

xgb_regressor = XGBRegressor()

xgb_search = GridSearchCV(estimator = xgb_regressor,
                          param_grid = xgb_grid,
                          cv = 5, 
                          verbose = 1, 
                          n_jobs = -1
                         )

xgb_search.fit(x_train, y_train)
print(xgb_search.best_params_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
{'learning_rate': 0.16, 'max_depth': 3, 'n_estimators': 40}


In [15]:
#number of trees used in sequence
trees_grid = [30,32,35,37,40,42,45]
# value of learning rate used by gradient descent
alpha_grid = [0.16, 0.17, 0.2, 0.25]
#maximum depth allowed for each tree
max_depth_grid = [2,3,4]


xgb_grid = {'n_estimators': trees_grid,
            'learning_rate': alpha_grid,
            'max_depth': max_depth_grid
            }

xgb_regressor = XGBRegressor()

xgb_search = GridSearchCV(estimator = xgb_regressor,
                          param_grid = xgb_grid,
                          cv = 5, 
                          verbose = 1, 
                          n_jobs = -1
                         )

xgb_search.fit(x_train, y_train)
print(xgb_search.best_params_)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
{'learning_rate': 0.16, 'max_depth': 3, 'n_estimators': 42}


In [16]:
 #number of trees used in sequence
trees_grid = [40,41,42,43,44,45]
# value of learning rate used by gradient descent
alpha_grid = [0.15, 0.16, 0.17]
#maximum depth allowed for each tree
max_depth_grid = [3,4]

xgb_grid = {'n_estimators': trees_grid,
            'learning_rate': alpha_grid,
            'max_depth': max_depth_grid
            }

xgb_regressor = XGBRegressor()

xgb_search = GridSearchCV(estimator = xgb_regressor,
                          param_grid = xgb_grid,
                          cv = 5, 
                          verbose = 1, 
                          n_jobs = -1
                         )

xgb_search.fit(x_train, y_train)
print(xgb_search.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'learning_rate': 0.16, 'max_depth': 3, 'n_estimators': 43}
